# PyTorch MNIST example dissected

In this notebook we'll explore the components of the
[PyTorch MNIST example](https://github.com/pytorch/examples/tree/master/mnist)
one-by-one.

**This is notebook 2 of the multi-part example.** Please see [Part 1 here](1_mnist_load.ipynb).

## 2. Building the model

In this notebook we'll explore the components of `torch.nn`

In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F

We'll start with copying the model from the original [PyTorch MNIST example](https://github.com/pytorch/examples/tree/master/mnist) verbatim:

In [3]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 10, kernel_size=5)
        self.conv2 = nn.Conv2d(10, 20, kernel_size=5)
        self.conv2_drop = nn.Dropout2d()
        self.fc1 = nn.Linear(320, 50)
        self.fc2 = nn.Linear(50, 10)

    def forward(self, x):
        x = F.relu(F.max_pool2d(self.conv1(x), 2))
        x = F.relu(F.max_pool2d(self.conv2_drop(self.conv2(x)), 2))
        x = x.view(-1, 320)
        x = F.relu(self.fc1(x))
        x = F.dropout(x, training=self.training)
        x = self.fc2(x)
        return F.log_softmax(x, dim=1)

That is, any class derived from `nn.Module` should implement the `.forward()` method; everything else is taken care of automagically by the auto-differentiation framework. Pretty neat.

The original `x` that comes into the `.forward()` method is the **batch** of features generated by the `DataLoader` (see [Part 1](1_mnist_load.ipynb#1.4-Dataset-and-DataLoader) for details). That is, for the MNIST dataset and batch size of 64, the dimensions of the input `x` will be

    torch.Size([64, 1, 28, 28])

First, we feed the input batch through the 2D convolution with 5x5 kernel and `10` output layers. Remember that our input has 1 layer, because it is greyscale; hence `1` in `Conv2d()` constructor parameters.